In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn import CrossEntropyLoss

In [2]:
from sklearn.decomposition import NMF
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
    
from scipy import sparse
from os import path

In [3]:
#Baselines

In [4]:
import pickle

file_path = 'jaccard_based_sim_ML1.pkl'

#with open(file_path, 'wb') as f:
    #pickle.dump(jaccard_based_sim, f)
with open(file_path, 'rb') as f:
    jaccard_based_sim = pickle.load(f)

In [5]:
import pickle

file_path = 'items_values_dict_ML1.pkl'

# Open the file in write binary mode and use pickle.dump to save the dictionary
#with open(file_path, 'wb') as f:
    #pickle.dump(items_values_dict, f)
with open(file_path, 'rb') as f:
    items_values_dict = pickle.load(f)    

In [6]:
import pickle

file_path = 'user_similarities_Jaccard_ML1.pkl'

# Open the file in write binary mode and use pickle.dump to save the dictionary
#with open(file_path, 'wb') as f:
    #pickle.dump(user_similarities_Jaccard, f)
with open(file_path, 'rb') as f:
    user_similarities_Jaccard = pickle.load(f)

In [7]:
import pickle

file_path = 'cosine_items_ML1.pkl'

# Open the file in write binary mode and use pickle.dump to save the dictionary
#with open(file_path, 'wb') as f:
    #pickle.dump(cosine_items_dict, f)
with open(file_path, 'rb') as f:
    cosine_items_dict = pickle.load(f)    

In [8]:
cosine_items = cosine_items_dict

In [9]:
import pickle

file_path = 'tf_idf_items_ML1.pkl'

# Open the file in write binary mode and use pickle.dump to save the dictionary
#with open(file_path, 'wb') as f:
    #pickle.dump(cosine_items_dict, f)
with open(file_path, 'rb') as f:
    tf_idf_items = pickle.load(f)  

In [10]:
import pickle

file_path = 'pop_dict.pkl'

with open(file_path, 'rb') as f:
    popularity_dict = pickle.load(f)  

In [11]:
import pickle

file_path = 'shap_values.pkl'

with open(file_path, 'rb') as f:
    shap_values = pickle.load(f)  

In [12]:
import pickle

file_path = 'item_to_cluster_ML1.pkl'

with open(file_path, 'rb') as f:
    item_to_cluster = pickle.load(f)  

In [13]:
shap_values.shape

(1208, 12)

In [14]:
##LIME

In [15]:
import importlib
import ipynb
from ipynb.fs.defs.lime import distance_to_proximity, LimeBase, get_lime_args, gaussian_kernel, mlp_get_lime_args
importlib.reload(ipynb.fs.defs.lime)
from ipynb.fs.defs.lime import distance_to_proximity, LimeBase, get_lime_args, gaussian_kernel, mlp_get_lime_args
'''
distance_to_proximity(distances_list) - takes distances from origin user and returns proximity
LimeBase() - class that gets kernel function
get_lime_args(user_vetor, item_id, model, items_array, min_pert = 10, max_pert = 20, num_of_perturbations = 5, seed = 0) - 
    returns neighborhood_data, neighborhood_labels, distances, item_id 
'''

lime = LimeBase(distance_to_proximity)

In [16]:
#Models

In [17]:
class MLP_G(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MLP_G, self).__init__()
        self.linear_x = nn.Linear(input_size, hidden_size, bias = False)
        self.linear_y = nn.Linear(input_size, hidden_size, bias = False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, user, item):
        user_representation = self.linear_x(user.float())
        item_representation = self.linear_y(item.float())
        dot_prod = torch.matmul(user_representation, item_representation.T)
        dot_sigmoid = self.sigmoid(dot_prod)
        
        return dot_sigmoid

In [18]:
class Recommender_G(nn.Module):
    def __init__(self, num_items, hidden_size):
        super(Recommender_G, self).__init__()
        self.mlp = MLP_G(num_items, hidden_size).to(device)

    def forward(self, user_vector, item_vector):
        user_vector = user_vector.to(device)
        item_vector = item_vector.to(device)
        output = self.mlp(user_vector, item_vector)
        return output.to(device)

In [19]:
class Explainer_G(nn.Module):
    def __init__(self, recommender_model_g, input_size, hidden_size):
        super(Explainer_G, self).__init__()
        
        backbone_children = list(recommender_model_g.children())[0]

        self.slice1 = nn.Sequential(*list(backbone_children.children())[:1])
        self.slice2 = nn.Sequential(*list(backbone_children.children())[1:2])
        self.bottleneck = nn.Sequential(
            nn.ReLU(),
            nn.Linear(in_features = hidden_size*2, out_features=input_size),
            nn.Sigmoid()
        ).to(device)

    def forward(self, user, item):
        slice1_output = self.slice1(user.float())
        slice2_output = self.slice2(item.float())
        combined_output = torch.cat((slice1_output, slice2_output), dim=-1)
        mask = self.bottleneck(combined_output).to(device)
        return mask

In [21]:
import torch.nn.functional as F
class LossModelCombined(torch.nn.Module):
    def __init__(self,alpha_parameter, recommender_model, explainer_model, hidden_size):
     
        super().__init__()
            
        self.recommender_model =  recommender_model
        self.explainer_model= explainer_model
        self.hidden_size = hidden_size
        self.alpha_parameter = alpha_parameter
        #self.beta_parameter = beta_parameter
        
    def forward(self,user_hist, y_positive, y_negative=None):
      
        user_hist = user_hist.to(device)
        y_positive = torch.tensor(y_positive).float().to(device)
        
        mask = self.explainer_model(user_hist,y_positive).to(device)
        x_masked = user_hist * mask
        y_positive_masked = self.recommender_model(x_masked,y_positive).unsqueeze(0).to(device)      
        pred_loss = -torch.log(y_positive_masked).to(device)
        
        pred_negative_loss = 0
        if y_negative is not None:
            y_negative_masked = self.recommender_model(x_masked,y_negative).unsqueeze(0).to(device) 
            pred_negative_loss = torch.log(y_negative_masked).to(device)

        
        mask_loss = torch.mean(torch.abs(mask)).to(device)
   
        comb_loss = pred_loss + self.alpha_parameter * mask_loss # + self.beta_parameter*pred_negative_loss 
        
        #print("Pred loss: {:.4f}, L1 loss: {:.4f}, Negative Loss {:.4f}".format(pred_loss.item(), mask_loss.item(), pred_negative_loss.item()))
        
        return x_masked, comb_loss  

In [22]:
##Train and test Data

In [23]:
# Train the model on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"
hidden_dim = 20
num_users = 6040
num_items = 3706
print("num_users is ", num_users)
print("num_items is ", num_items)

num_users is  6040
num_items is  3706


In [24]:
rec_model = Recommender_G(num_items, hidden_dim)
rec_model.load_state_dict(torch.load('recommender_model.pt'))

<All keys matched successfully>

In [25]:
for param in rec_model.parameters():
    param.requires_grad= False

In [26]:
##Read data from the files

In [27]:
train_data_mixed = pd.read_csv('train_data_mixed.csv')

In [28]:
test_data = pd.read_csv('test_data.csv')

In [29]:
import pickle
with open('items_values_dict_ML1.pkl', 'rb') as f:
    items_values_dict = pickle.load(f) # deserialize using load()

In [30]:
import pickle
with open('prob_dict.pkl', 'rb') as f:
    prob_dict = pickle.load(f) # deserialize using load()

In [31]:
items_values= pd.read_csv('items_values.csv')

In [32]:
items_array = items_values.to_numpy()

In [33]:
train_array = train_data_mixed.to_numpy()
test_array = test_data.to_numpy()

In [34]:
#Top k dictionaries for train and test

In [35]:
filename = 'topk_train.pkl'

# open the file in write-binary mode and save the array
with open(filename, 'rb') as f:
    topk_train = pickle.load(f)

In [36]:
filename = 'topk_test.pkl'

# open the file in write-binary mode and save the array
with open(filename, 'rb') as f:
    topk_test = pickle.load(f)

In [37]:
##Metrics calculation

In [38]:
def get_top_k(user_vector, original_user_vector, num_items, model, top_k):
    item_prob_dict = {}
    user_tensor = torch.Tensor(user_vector).to(device)
    item_tensor = torch.FloatTensor(items_array).to(device)
    output_model = [float(i) for i in model(user_tensor, item_tensor).cpu().detach().numpy()]
    
    original_user_vector = np.array(original_user_vector.cpu())
    neg = np.ones_like(original_user_vector)- original_user_vector
    output = neg*output_model
    for i in range(len(output)):
        item_prob_dict[i]=output[i]

    sorted_items_by_prob  = sorted(item_prob_dict.items(), key=lambda item: item[1],reverse=True)

    return dict(sorted_items_by_prob[0:top_k])

In [39]:
def get_index_in_the_list(user_vector, original_user_vector, item_id, num_items, model):
    top_k_list = list(get_top_k(user_vector, original_user_vector, num_items, model, num_items).keys())
    return top_k_list.index(item_id)

In [40]:
import warnings
warnings.filterwarnings('ignore')

In [41]:
def get_ndcg(ranked_list, target_item):
    if target_item not in ranked_list:
        return 0.0

    target_idx = torch.tensor(ranked_list.index(target_item), device=device)
    ndcg = torch.reciprocal(torch.log2(target_idx + 2))

    return ndcg.item()

In [42]:
def single_user_metrics(user_vector, item_id, k, num_of_bins, num_items, recommender_model, model_combined, y_pred = None, mask_type = None, user_id = None):

    
    user_tensor = torch.FloatTensor(user_vector).to(device)
    
    item_vector = items_values_dict[item_id]
    item_tensor = torch.FloatTensor(item_vector).to(device)
    
    POS_masked = user_tensor
    NEG_masked = user_tensor
    POS_masked[item_id]=0
    NEG_masked[item_id]=0
    
    user_hist_size = np.sum(user_vector)

    
    bins=[0]+[len(x) for x in np.array_split(np.arange(np.sum(user_vector)), num_of_bins, axis=0)]
    
    POS_at_1 = [0]*(len(bins))
    POS_at_5 = [0]*(len(bins))
    POS_at_10=[0]*(len(bins))
    POS_at_20=[0]*(len(bins))
    POS_at_50=[0]*(len(bins))
    POS_at_100=[0]*(len(bins))
    
    NEG_at_1 = [0]*(len(bins))
    NEG_at_5 = [0]*(len(bins))
    NEG_at_10 = [0]*(len(bins))
    NEG_at_20 = [0]*(len(bins))
    NEG_at_50 = [0]*(len(bins))
    NEG_at_100 = [0]*(len(bins))
    
    DEL = [0]*(len(bins))
    INS = [0]*(len(bins))
    
    rankA_at_1 = [0]*(len(bins))
    rankA_at_5 = [0]*(len(bins))
    rankA_at_10 = [0]*(len(bins))
    rankA_at_20 = [0]*(len(bins))
    rankA_at_50 = [0]*(len(bins))
    rankA_at_100 = [0]*(len(bins))
    
    rankB = [0]*(len(bins))
    
    NDCG_at_1 = [0]*(len(bins))
    NDCG_at_5 = [0]*(len(bins))
    NDCG_at_10 = [0]*(len(bins))
    NDCG_at_20 = [0]*(len(bins))
    NDCG_at_50 = [0]*(len(bins))
    NDCG_at_100 = [0]*(len(bins))
    
    total_items = 0
    
    for i in range(len(bins)):
        total_items += bins[i]
        
        if i == 0:
            if mask_type == 'jaccard_g':
                sim_items = find_jaccard_g_mask(POS_masked, item_id, num_items, jaccard_based_sim , num_samples=bins[i])
            elif mask_type == 'jaccard_u':
                sim_items = find_jaccard_u_mask(POS_masked, item_id, num_items, user_similarities_Jaccard , num_samples=bins[i])
            elif mask_type == 'cosine':
                sim_items = find_cosine_mask(POS_masked, item_id, num_items, cosine_items , num_samples=bins[i])
            elif mask_type == 'tf_idf':
                sim_items = find_tf_idf_mask(POS_masked, item_id, num_items, tf_idf_items)
            elif mask_type == 'pop':
                sim_items = find_pop_mask(POS_masked, item_id, num_items , num_samples=bins[i])
            elif mask_type == 'shap': 
                sim_items = find_shapley_mask(POS_masked, user_id, recommender_model, shap_values, item_to_cluster)
            elif mask_type == 'lime':
                sim_items = find_LIME_mask(user_vector, item_id, items_array, 50, 100, 150, distance_to_proximity,'highest_weights', model = recommender_model, num_samples=user_hist_size)
            elif mask_type == 'ltx':
                sim_items = find_LTX_mask(POS_masked, item_vector, item_id, model_combined)
            else:
                raise Exception("Wrong mask type")
        
        POS_sim_items  = list(sorted(sim_items.items(), key=lambda item: item[1],reverse=True))[0:user_hist_size]
        NEG_sim_items  = list(sorted(dict(POS_sim_items).items(), key=lambda item: item[1],reverse=False))
        
        POS_masked = torch.zeros_like(user_tensor, dtype=torch.float32, device=device)
        for j in POS_sim_items[:total_items]:
            POS_masked[j[0]] = 1
        POS_masked = user_tensor - POS_masked
        
        NEG_masked = torch.zeros_like(user_tensor, dtype=torch.float32, device=device)
        for j in NEG_sim_items[:total_items]:
            NEG_masked[j[0]] = 1
        NEG_masked = user_tensor - NEG_masked # remove the masked items from the user history 

        POS_ranked_list = get_top_k(POS_masked, user_tensor,num_items, recommender_model,num_items)        
        POS_index = list(POS_ranked_list.keys()).index(item_id)+1
        NEG_index = get_index_in_the_list(NEG_masked,user_tensor, item_id, num_items, recommender_model)+1
   
        # for pos:
        POS_at_1[i] = 1 if POS_index <=1 else 0
        POS_at_5[i] = 1 if POS_index <=5 else 0
        POS_at_10[i] = 1 if POS_index <=10 else 0
        POS_at_20[i] = 1 if POS_index <=20 else 0
        POS_at_50[i] = 1 if POS_index <=50 else 0
        POS_at_100[i] = 1 if POS_index <=100 else 0

        # for neg:
        NEG_at_1[i] = 1 if NEG_index <=1 else 0
        NEG_at_5[i] = 1 if NEG_index <=5 else 0
        NEG_at_10[i] = 1 if NEG_index <=10 else 0
        NEG_at_20[i] = 1 if NEG_index <=20 else 0
        NEG_at_50[i] = 1 if NEG_index <=50 else 0
        NEG_at_100[i] = 1 if NEG_index <=100 else 0
        # for del:
        DEL[i] = float(recommender_model(POS_masked, item_tensor).detach().cpu().numpy())
        
        # for ins:
        INS[i] = float(recommender_model(user_tensor-POS_masked, item_tensor).detach().cpu().numpy())
        
       # for rankA:
        rankA_at_1[i] = max(0, (1+1-POS_index)/10)
        rankA_at_5[i] = max(0, (5+1-POS_index)/20)
        rankA_at_10[i] = max(0, (10+1-POS_index)/10)
        rankA_at_20[i] = max(0, (20+1-POS_index)/20)
        rankA_at_50[i] = max(0, (50+1-POS_index)/50)
        rankA_at_100[i] = max(0, (100+1-POS_index)/100)

        # for rankB:
        rankB[i] = 1/POS_index

        #for NDCG:
        NDCG_at_1[i]= get_ndcg(list(POS_ranked_list.keys())[:1],item_id)
        NDCG_at_5[i]= get_ndcg(list(POS_ranked_list.keys())[:5],item_id)
        NDCG_at_10[i]= get_ndcg(list(POS_ranked_list.keys())[:10],item_id)
        NDCG_at_20[i]= get_ndcg(list(POS_ranked_list.keys())[:20],item_id)
        NDCG_at_50[i]= get_ndcg(list(POS_ranked_list.keys())[:50],item_id)
        NDCG_at_100[i]= get_ndcg(list(POS_ranked_list.keys())[:100],item_id)

    res = [POS_at_1, POS_at_5, POS_at_10, POS_at_20, POS_at_50, POS_at_100, NEG_at_1, NEG_at_5, NEG_at_10, NEG_at_20, NEG_at_50, NEG_at_100, DEL, INS, rankA_at_1, rankA_at_5, rankA_at_10, rankA_at_20, rankA_at_50, rankA_at_100, rankB, NDCG_at_1, NDCG_at_5, NDCG_at_10, NDCG_at_20, NDCG_at_50, NDCG_at_100]
    for i in range(len(res)):
        res[i] = np.array(res[i])
        
    return res

In [44]:
#Mask calculation

In [45]:
def find_LIME_mask(x, item_id, items_array, min_pert, max_pert, num_of_perturbations, kernel_func, feature_selection, model, num_samples=10, method = 'POS'):
    
    user_hist = x 
    # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    lime.kernel_fn = kernel_func
    neighborhood_data, neighborhood_labels, distances, item_id = mlp_get_lime_args(user_hist, item_id, model, items_array, min_pert = min_pert, max_pert = max_pert, num_of_perturbations = num_of_perturbations, seed = item_id)
  
    most_pop_items  = lime.explain_instance_with_data(neighborhood_data, neighborhood_labels, distances, item_id, num_samples, feature_selection, pos_neg='POS')
    most_pop_items_dict =  {key: value for key, value in most_pop_items}
    
    return most_pop_items_dict

In [46]:
#LTX based similarity
def find_LTX_mask(x, y_true, item_id, model_combined):
    
    user_hist = torch.tensor(x) 
    user_hist[item_id] = 0 

    x_masked_g, loss_comb_g = model_combined(user_hist, y_true)
    
    x_masked_g = x_masked_g.to(device)
    #item_sim_dict = {i: v for i, v in enumerate(x_masked_l)}
    item_sim_dict = {i: x_masked_g[i].item() for i in range(x_masked_g.numel())}
    
    return item_sim_dict

In [47]:
#Genre based similarities using Jaccard
def find_jaccard_g_mask(x, item_id, num_items,  jaccard_based_sim,num_samples=10, method = 'POS'):
    user_hist = torch.tensor(x) # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    item_jaccard_dict = {}
    for i in range(num_items): 
        if(user_hist[i] == 1): # iterate all positive items of the user
            if((i,item_id) in jaccard_based_sim.keys()):
                item_jaccard_dict[i]=jaccard_based_sim[(i,item_id)] # add Jaccard similarity between items
            else:
                item_jaccard_dict[i] = 0
    
    return item_jaccard_dict

In [48]:
#User based similarities using Jaccard
def find_jaccard_u_mask(x, item_id, num_items, user_based_Jaccard_sim,num_samples=10, method = 'POS'):
    user_hist = torch.tensor(x) # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    item_jaccard_dict = {}
    for i in range(num_items): 
        if(user_hist[i] == 1): # iterate all positive items of the user
            if((i,item_id) in user_based_Jaccard_sim.keys()):
                item_jaccard_dict[i]=user_based_Jaccard_sim[(i,item_id)] # add Jaccard similarity between items
            else:
                item_jaccard_dict[i] = 0
            
    return item_jaccard_dict

In [49]:
#Cosine based similarities between users and items
def find_cosine_mask(x, item_id, num_items, item_cosine, num_samples=10, method = 'POS'):

    user_hist = torch.tensor(x) # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    item_cosine_dict = {}
        
    for i in range(num_items): 
        if(user_hist[i] == 1): # iterate all positive items of the user
            if((i,item_id) in item_cosine.keys()):
                item_cosine_dict[i]=item_cosine[(i,item_id)] # add cosine similarity between items
            else:
                item_cosine_dict[i] = 0
    
    return item_cosine_dict

In [50]:
#tf_idf mask
def find_tf_idf_mask(x, item_id, num_items, tf_idf_sim):
    
    x = x.cpu().detach().numpy()
    x[item_id] = 0
    
    positive_items = np.where(x == 1)[0]
    tf_idf_dict = {i: tf_idf_sim.get((i, item_id), 0) for i in positive_items}
    
    return tf_idf_dict

In [51]:
#popularity mask
def find_pop_mask(x, item_id, num_items, num_samples=10, method = 'POS'):
    user_hist = torch.tensor(x) # remove the positive item we want to explain from the user history
    user_hist[item_id] = 0
    item_pop_dict = {}
    for i in range(num_items): 
        if(user_hist[i] == 1): # iterate over all positive items of the user
            item_pop_dict[i]=popularity_dict[i] # add the pop of the item to the dictionary
            
            
    return item_pop_dict

In [52]:
def find_shapley_mask(user_vector, user_id, model, shap_values, item_to_cluster):
        
        item_shap = {}
        shapley_values = shap_values[shap_values[:, 0].astype(int) == user_id][:,2:]
        user_vector = user_vector.cpu().detach().numpy().astype(int)
        
        for i in np.where(user_vector.astype(int) == 1)[0]:
            items_cluster = item_to_cluster[i]
            item_shap[i] = shapley_values.T[int(items_cluster)][0]
  
        return item_shap     

In [48]:
#LTX train

In [54]:
#Get users vectors to create topk
unique_indices = np.unique(train_array[:,-3], return_index=True, axis=0)[1]

# create a new array with only the unique users
train_unique_arr = train_array[unique_indices, :]

In [53]:
explainer_model_g = Explainer_G(rec_model, num_items, hidden_dim).to(device) 

In [55]:
#LTX training

import time
import random
train_losses = []
epochs = 10
hidden_dim = 10
num_of_bins = 10
alpha_parameter = 0.5
#beta_parameter = 0.2

model_combined = LossModelCombined(alpha_parameter, rec_model, explainer_model_g, hidden_dim).to(device)
optimizer_comb = torch.optim.Adam(model_combined.parameters(), lr=0.04)
    
#Train LTX
for epoch in range(epochs):
    train_loss = 0
    
    for i in range(train_unique_arr.shape[0]):
        
        #user data
        user_id = train_unique_arr[i][-3]
        user_vector = train_unique_arr[i][:-3]
        #get top1 of this user for LTX training
        top1_item = np.argmax(topk_train[user_id])
        #print("top1_item ", top1_item)
        #Get negative item
        #top_indices = np.argsort(list(topk_train[user_id]))[-100:-1]
        # Sample one index randomly
        #top_100_indices = np.delete(top_indices, np.where(top_indices == top1_item))
    
        # sample an index from the top 100
        #not_top1 = np.random.choice(top_100_indices)
        #print(not_top1)
        #print("not_top1_item ", not_top1)
       
        positive_sample = torch.FloatTensor(items_values_dict[top1_item]).to(device)
        #negative_sample = torch.FloatTensor(items_values_dict[not_top1]).to(device)
        
        user_vector[top1_item] = 0 
        user_tensor = torch.FloatTensor(user_vector).to(device)

        #train model        
        optimizer_comb.zero_grad()
        x_masked_g, loss_comb_g = model_combined(user_tensor, positive_sample)#, negative_sample)
        
        train_loss+=loss_comb_g.item()
            
        loss_comb_g.backward()
        optimizer_comb.step()
   
    train_losses.append(train_loss/train_unique_arr.shape[0])

    print(f"Epoch {epoch}, Train Loss {train_loss/train_unique_arr.shape[0]:.4f}")

Epoch 0, Train Loss 0.1002
Epoch 1, Train Loss 0.0640
Epoch 2, Train Loss 0.0503
Epoch 3, Train Loss 0.0433
Epoch 4, Train Loss 0.0375
Epoch 5, Train Loss 0.0331
Epoch 6, Train Loss 0.0358
Epoch 7, Train Loss 0.0312
Epoch 8, Train Loss 0.0298
Epoch 9, Train Loss 0.0298


In [138]:
torch.save(model_combined.state_dict(), 'mlp_model_combined_10_epochs_05.pt')

In [139]:
torch.save(explainer_model_g.state_dict(), 'mlp_explainer_model_10_epochs_05.pt')

In [ ]:
#LTX on testing data set

In [61]:
import time
# Evaluate the model on the test set
num_of_bins = 11
k = 100


POS_at_1_j_g = np.zeros(num_of_bins)
POS_at_5_j_g = np.zeros(num_of_bins)
POS_at_10_j_g = np.zeros(num_of_bins)
POS_at_20_j_g = np.zeros(num_of_bins)
POS_at_50_j_g = np.zeros(num_of_bins)
POS_at_100_j_g = np.zeros(num_of_bins)
NEG_at_1_j_g = np.zeros(num_of_bins)
NEG_at_5_j_g = np.zeros(num_of_bins)
NEG_at_10_j_g = np.zeros(num_of_bins)
NEG_at_20_j_g = np.zeros(num_of_bins)
NEG_at_50_j_g = np.zeros(num_of_bins)
NEG_at_100_j_g = np.zeros(num_of_bins)
users_DEL_j_g = np.zeros(num_of_bins)
users_INS_j_g = np.zeros(num_of_bins)
rank_at_1_j_g = np.zeros(num_of_bins)
rank_at_5_j_g = np.zeros(num_of_bins)
rank_at_10_A_j_g = np.zeros(num_of_bins)
rank_at_20_A_j_g = np.zeros(num_of_bins)
rank_at_50_A_j_g = np.zeros(num_of_bins)
rank_at_100_A_j_g = np.zeros(num_of_bins)
rank_at_k_B_j_g = np.zeros(num_of_bins)
NDCG_at_1_j_g = np.zeros(num_of_bins)
NDCG_at_5_j_g = np.zeros(num_of_bins)
NDCG_at_10_j_g = np.zeros(num_of_bins)
NDCG_at_20_j_g = np.zeros(num_of_bins)
NDCG_at_50_j_g = np.zeros(num_of_bins)
NDCG_at_100_j_g = np.zeros(num_of_bins)

POS_at_1_ltx = np.zeros(num_of_bins)
POS_at_5_ltx = np.zeros(num_of_bins)
POS_at_10_ltx = np.zeros(num_of_bins)
POS_at_20_ltx = np.zeros(num_of_bins)
POS_at_50_ltx = np.zeros(num_of_bins)
POS_at_100_ltx = np.zeros(num_of_bins)
NEG_at_1_ltx = np.zeros(num_of_bins)
NEG_at_5_ltx = np.zeros(num_of_bins)
NEG_at_10_ltx = np.zeros(num_of_bins)
NEG_at_20_ltx = np.zeros(num_of_bins)
NEG_at_50_ltx = np.zeros(num_of_bins)
NEG_at_100_ltx = np.zeros(num_of_bins)
users_DEL_ltx = np.zeros(num_of_bins)
users_INS_ltx = np.zeros(num_of_bins)
rank_at_1_ltx = np.zeros(num_of_bins)
rank_at_5_ltx = np.zeros(num_of_bins)
rank_at_10_A_ltx = np.zeros(num_of_bins)
rank_at_20_A_ltx = np.zeros(num_of_bins)
rank_at_50_A_ltx = np.zeros(num_of_bins)
rank_at_100_A_ltx = np.zeros(num_of_bins)
rank_at_k_B_ltx = np.zeros(num_of_bins)
NDCG_at_1_ltx = np.zeros(num_of_bins)
NDCG_at_5_ltx = np.zeros(num_of_bins)
NDCG_at_10_ltx = np.zeros(num_of_bins)
NDCG_at_20_ltx = np.zeros(num_of_bins)
NDCG_at_50_ltx = np.zeros(num_of_bins)
NDCG_at_100_ltx = np.zeros(num_of_bins)

POS_at_1_shap = np.zeros(num_of_bins)
POS_at_5_shap = np.zeros(num_of_bins)
POS_at_10_shap = np.zeros(num_of_bins)
POS_at_20_shap = np.zeros(num_of_bins)
POS_at_50_shap = np.zeros(num_of_bins)
POS_at_100_shap = np.zeros(num_of_bins)
NEG_at_1_shap = np.zeros(num_of_bins)
NEG_at_5_shap = np.zeros(num_of_bins)
NEG_at_10_shap = np.zeros(num_of_bins)
NEG_at_20_shap = np.zeros(num_of_bins)
NEG_at_50_shap = np.zeros(num_of_bins)
NEG_at_100_shap = np.zeros(num_of_bins)
users_DEL_shap = np.zeros(num_of_bins)
users_INS_shap = np.zeros(num_of_bins)
rank_at_1_shap = np.zeros(num_of_bins)
rank_at_5_shap = np.zeros(num_of_bins)
rank_at_10_A_shap = np.zeros(num_of_bins)
rank_at_20_A_shap = np.zeros(num_of_bins)
rank_at_50_A_shap = np.zeros(num_of_bins)
rank_at_100_A_shap = np.zeros(num_of_bins)
rank_at_k_B_shap = np.zeros(num_of_bins)
NDCG_at_1_shap = np.zeros(num_of_bins)
NDCG_at_5_shap = np.zeros(num_of_bins)
NDCG_at_10_shap = np.zeros(num_of_bins)
NDCG_at_20_shap = np.zeros(num_of_bins)
NDCG_at_50_shap = np.zeros(num_of_bins)
NDCG_at_100_shap = np.zeros(num_of_bins)

num_of_bins = 10

explainer_model_g.eval()
model_combined.eval()

with torch.no_grad():
    for i in range(test_array.shape[0]):

        if i%500 == 0:
            print(i)
        #item_id = test_array[i][-1]
        user_id = test_array[i][-2]

        #item_vector = items_values.iloc[item_id]
        user_vector = test_array[i][:-2]

        y_pred = topk_test[user_id]
        item_id = np.argmax(y_pred)

        item_vector = items_values_dict[item_id]
        item_tensor = torch.FloatTensor(item_vector).to(device)

        user_vector[item_id] = 0
        user_tensor = torch.FloatTensor(user_vector).to(device)
     
       

         ### Lime:
        res = single_user_metrics(user_vector, item_id, k, num_of_bins, num_items, rec_model, model_combined, user_id = user_id, mask_type= 'shap')    
        POS_at_1_shap += res[0]
        POS_at_5_shap += res[1]
        POS_at_10_shap += res[2]
        POS_at_20_shap += res[3]
        POS_at_50_shap += res[4]
        POS_at_100_shap += res[5]
        NEG_at_1_shap += res[6]
        NEG_at_5_shap += res[7]
        NEG_at_10_shap += res[8]
        NEG_at_20_shap += res[9]
        NEG_at_50_shap += res[10]
        NEG_at_100_shap += res[11]
        users_DEL_shap += res[12]
        users_INS_shap += res[13]
        rank_at_1_shap += res[14]
        rank_at_5_shap += res[15]
        rank_at_10_A_shap += res[16]
        rank_at_20_A_shap += res[17]
        rank_at_50_A_shap += res[18]
        rank_at_100_A_shap += res[19]
        rank_at_k_B_shap += res[20]
        NDCG_at_1_shap += res[21]
        NDCG_at_5_shap += res[22]
        NDCG_at_10_shap += res[23]
        NDCG_at_20_shap += res[24]
        NDCG_at_50_shap += res[25]
        NDCG_at_100_shap += res[26]
        
      
        if(i%100 == 0):
            print(i)
           
a = i+1

print('POS_at_1_j_g: ', np.mean(POS_at_1_j_g[1:])/a)
print('POS_at_5_j_g: ', np.mean(POS_at_5_j_g[1:])/a)
print('POS_at_10_j_g: ', np.mean(POS_at_10_j_g[1:])/a)
print('POS_at_20_j_g: ', np.mean(POS_at_20_j_g[1:])/a)
print('POS_at_50_j_g: ', np.mean(POS_at_50_j_g[1:])/a)
print('POS_at_100_j_g: ', np.mean(POS_at_100_j_g[1:])/a)
print('NEG_at_1_j_g: ', np.mean(NEG_at_1_j_g[1:])/a)
print('NEG_at_5_j_g: ', np.mean(NEG_at_5_j_g[1:])/a)
print('NEG_at_10_j_g: ', np.mean(NEG_at_10_j_g[1:])/a)
print('NEG_at_20_j_g: ', np.mean(NEG_at_20_j_g[1:])/a)
print('NEG_at_50_j_g: ', np.mean(NEG_at_50_j_g[1:])/a)
print('NEG_at_100_j_g: ', np.mean(NEG_at_100_j_g[1:])/a)
print('users_DEL_j_g: ', np.mean(users_DEL_j_g[1:])/a)
print('users_INS_j_g: ', np.mean(users_INS_j_g[1:])/a)
print('rank_at_1_j_g: ', np.mean(rank_at_1_j_g[1:])/a)
print('rank_at_5_j_g: ', np.mean(rank_at_5_j_g[1:])/a)
print('rank_at_10_A_j_g: ', np.mean(rank_at_10_A_j_g[1:])/a)
print('rank_at_20_A_j_g: ', np.mean(rank_at_20_A_j_g[1:])/a)
print('rank_at_50_A_j_g: ', np.mean(rank_at_50_A_j_g[1:])/a)
print('rank_at_100_A_j_g: ', np.mean(rank_at_100_A_j_g[1:])/a)
print('rank_at_k_B_j_g: ', np.mean(rank_at_k_B_j_g[1:])/a)
print('NDCG_at_1_j_g: ', np.mean(NDCG_at_1_j_g[1:])/a)
print('NDCG_at_5_j_g: ', np.mean(NDCG_at_5_j_g[1:])/a)
print('NDCG_at_10_j_g: ', np.mean(NDCG_at_10_j_g[1:])/a)
print('NDCG_at_20_j_g: ', np.mean(NDCG_at_20_j_g[1:])/a)
print('NDCG_at_50_j_g: ', np.mean(NDCG_at_50_j_g[1:])/a)
print('NDCG_at_100_j_g: ', np.mean(NDCG_at_100_j_g[1:])/a)

print('POS_at_1_ltx: ', np.mean(POS_at_1_ltx[1:])/a)
print('POS_at_5_ltx: ', np.mean(POS_at_5_ltx[1:])/a)
print('POS_at_10_ltx: ', np.mean(POS_at_10_ltx[1:])/a)
print('POS_at_20_ltx: ', np.mean(POS_at_20_ltx[1:])/a)
print('POS_at_50_ltx: ', np.mean(POS_at_50_ltx[1:])/a)
print('POS_at_100_ltx: ', np.mean(POS_at_100_ltx[1:])/a)
print('NEG_at_1_ltx: ', np.mean(NEG_at_1_ltx[1:])/a)
print('NEG_at_5_ltx: ', np.mean(NEG_at_5_ltx[1:])/a)
print('NEG_at_10_ltx: ', np.mean(NEG_at_10_ltx[1:])/a)
print('NEG_at_20_ltx: ', np.mean(NEG_at_20_ltx[1:])/a)
print('NEG_at_50_ltx: ', np.mean(NEG_at_50_ltx[1:])/a)
print('NEG_at_100_ltx: ', np.mean(NEG_at_100_ltx[1:])/a)
print('users_DEL_ltx: ', np.mean(users_DEL_ltx[1:])/a)
print('users_INS_ltx: ', np.mean(users_INS_ltx[1:])/a)
print('rank_at_1_ltx: ', np.mean(rank_at_1_ltx[1:])/a)
print('rank_at_5_ltx: ', np.mean(rank_at_5_ltx[1:])/a)
print('rank_at_10_A_ltx: ', np.mean(rank_at_10_A_ltx[1:])/a)
print('rank_at_20_A_ltx: ', np.mean(rank_at_20_A_ltx[1:])/a)
print('rank_at_50_A_ltx: ', np.mean(rank_at_50_A_ltx[1:])/a)
print('rank_at_100_A_ltx: ', np.mean(rank_at_100_A_ltx[1:])/a)
print('rank_at_k_B_ltx: ', np.mean(rank_at_k_B_ltx[1:])/a)
print('NDCG_at_1_ltx: ', np.mean(NDCG_at_1_ltx[1:])/a)
print('NDCG_at_5_ltx: ', np.mean(NDCG_at_5_ltx[1:])/a)
print('NDCG_at_10_ltx: ', np.mean(NDCG_at_10_ltx[1:])/a)
print('NDCG_at_20_ltx: ', np.mean(NDCG_at_20_ltx[1:])/a)
print('NDCG_at_50_ltx: ', np.mean(NDCG_at_50_ltx[1:])/a)
print('NDCG_at_100_ltx: ', np.mean(NDCG_at_100_ltx[1:])/a)

print('POS_at_1_shap: ', np.mean(POS_at_1_shap[1:])/a)
print('POS_at_5_shap: ', np.mean(POS_at_5_shap[1:])/a)
print('POS_at_10_shap: ', np.mean(POS_at_10_shap[1:])/a)
print('POS_at_20_shap: ', np.mean(POS_at_20_shap[1:])/a)
print('POS_at_50_shap: ', np.mean(POS_at_50_shap[1:])/a)
print('POS_at_100_shap: ', np.mean(POS_at_100_shap[1:])/a)
print('NEG_at_1_shap: ', np.mean(NEG_at_1_shap[1:])/a)
print('NEG_at_5_shap: ', np.mean(NEG_at_5_shap[1:])/a)
print('NEG_at_10_shap: ', np.mean(NEG_at_10_shap[1:])/a)
print('NEG_at_20_shap: ', np.mean(NEG_at_20_shap[1:])/a)
print('NEG_at_50_shap: ', np.mean(NEG_at_50_shap[1:])/a)
print('NEG_at_100_shap: ', np.mean(NEG_at_100_shap[1:])/a)
print('users_DEL_shap: ', np.mean(users_DEL_shap[1:])/a)
print('users_INS_shap: ', np.mean(users_INS_shap[1:])/a)
print('rank_at_1_shap: ', np.mean(rank_at_1_shap[1:])/a)
print('rank_at_5_shap: ', np.mean(rank_at_5_shap[1:])/a)
print('rank_at_10_A_shap: ', np.mean(rank_at_10_A_shap[1:])/a)
print('rank_at_20_A_shap: ', np.mean(rank_at_20_A_shap[1:])/a)
print('rank_at_50_A_shap: ', np.mean(rank_at_50_A_shap[1:])/a)
print('rank_at_100_A_shap: ', np.mean(rank_at_100_A_shap[1:])/a)
print('rank_at_k_B_shap: ', np.mean(rank_at_k_B_shap[1:])/a)
print('NDCG_at_1_shap: ', np.mean(NDCG_at_1_shap[1:])/a)
print('NDCG_at_5_shap: ', np.mean(NDCG_at_5_shap[1:])/a)
print('NDCG_at_10_shap: ', np.mean(NDCG_at_10_shap[1:])/a)
print('NDCG_at_20_shap: ', np.mean(NDCG_at_50_shap[1:])/a)
print('NDCG_at_100_shap: ', np.mean(NDCG_at_100_shap[1:])/a)

0
0
100
200
300
400
500
500
600
700
800
900
1000
1000
1100
1200
POS_at_1_j_g:  0.0
POS_at_5_j_g:  0.0
POS_at_10_j_g:  0.0
POS_at_20_j_g:  0.0
POS_at_50_j_g:  0.0
POS_at_100_j_g:  0.0
NEG_at_1_j_g:  0.0
NEG_at_5_j_g:  0.0
NEG_at_10_j_g:  0.0
NEG_at_20_j_g:  0.0
NEG_at_50_j_g:  0.0
NEG_at_100_j_g:  0.0
users_DEL_j_g:  0.0
users_INS_j_g:  0.0
rank_at_1_j_g:  0.0
rank_at_5_j_g:  0.0
rank_at_10_A_j_g:  0.0
rank_at_20_A_j_g:  0.0
rank_at_50_A_j_g:  0.0
rank_at_100_A_j_g:  0.0
rank_at_k_B_j_g:  0.0
NDCG_at_1_j_g:  0.0
NDCG_at_5_j_g:  0.0
NDCG_at_10_j_g:  0.0
NDCG_at_20_j_g:  0.0
NDCG_at_50_j_g:  0.0
NDCG_at_100_j_g:  0.0
POS_at_1_ltx:  0.0
POS_at_5_ltx:  0.0
POS_at_10_ltx:  0.0
POS_at_20_ltx:  0.0
POS_at_50_ltx:  0.0
POS_at_100_ltx:  0.0
NEG_at_1_ltx:  0.0
NEG_at_5_ltx:  0.0
NEG_at_10_ltx:  0.0
NEG_at_20_ltx:  0.0
NEG_at_50_ltx:  0.0
NEG_at_100_ltx:  0.0
users_DEL_ltx:  0.0
users_INS_ltx:  0.0
rank_at_1_ltx:  0.0
rank_at_5_ltx:  0.0
rank_at_10_A_ltx:  0.0
rank_at_20_A_ltx:  0.0
rank_at_50_A_l

In [63]:
print('POS_at_1_shap: ', np.mean(POS_at_1_shap[1:])/a)
print('POS_at_5_shap: ', np.mean(POS_at_5_shap[1:])/a)
print('POS_at_10_shap: ', np.mean(POS_at_10_shap[1:])/a)
print('POS_at_20_shap: ', np.mean(POS_at_20_shap[1:])/a)
print('POS_at_50_shap: ', np.mean(POS_at_50_shap[1:])/a)
print('POS_at_100_shap: ', np.mean(POS_at_100_shap[1:])/a)
print('NEG_at_1_shap: ', np.mean(NEG_at_1_shap[1:])/a)
print('NEG_at_5_shap: ', np.mean(NEG_at_5_shap[1:])/a)
print('NEG_at_10_shap: ', np.mean(NEG_at_10_shap[1:])/a)
print('NEG_at_20_shap: ', np.mean(NEG_at_20_shap[1:])/a)
print('NEG_at_50_shap: ', np.mean(NEG_at_50_shap[1:])/a)
print('NEG_at_100_shap: ', np.mean(NEG_at_100_shap[1:])/a)
print('users_DEL_shap: ', np.mean(users_DEL_shap[1:])/a)
print('users_INS_shap: ', np.mean(users_INS_shap[1:])/a)
print('rank_at_1_shap: ', np.mean(rank_at_1_shap[1:])/a)
print('rank_at_5_shap: ', np.mean(rank_at_5_shap[1:])/a)
print('rank_at_10_A_shap: ', np.mean(rank_at_10_A_shap[1:])/a)
print('rank_at_20_A_shap: ', np.mean(rank_at_20_A_shap[1:])/a)
print('rank_at_50_A_shap: ', np.mean(rank_at_50_A_shap[1:])/a)
print('rank_at_100_A_shap: ', np.mean(rank_at_100_A_shap[1:])/a)
print('rank_at_k_B_shap: ', np.mean(rank_at_k_B_shap[1:])/a)
print('NDCG_at_1_shap: ', np.mean(NDCG_at_1_shap[1:])/a)
print('NDCG_at_5_shap: ', np.mean(NDCG_at_5_shap[1:])/a)
print('NDCG_at_10_shap: ', np.mean(NDCG_at_10_shap[1:])/a)
print('NDCG_at_20_shap: ', np.mean(NDCG_at_20_shap[1:])/a)
print('NDCG_at_50_shap: ', np.mean(NDCG_at_50_shap[1:])/a)
print('NDCG_at_100_shap: ', np.mean(NDCG_at_100_shap[1:])/a)

POS_at_1_shap:  0.24710264900662252
POS_at_5_shap:  0.35587748344370856
POS_at_10_shap:  0.4147350993377483
POS_at_20_shap:  0.4709437086092715
POS_at_50_shap:  0.5795529801324504
POS_at_100_shap:  0.6733443708609271
NEG_at_1_shap:  0.22996688741721855
NEG_at_5_shap:  0.3348509933774834
NEG_at_10_shap:  0.38874172185430467
NEG_at_20_shap:  0.4432119205298013
NEG_at_50_shap:  0.5507450331125827
NEG_at_100_shap:  0.6437086092715232
users_DEL_shap:  0.8846360638735905
users_INS_shap:  0.9343646848311186
rank_at_1_shap:  0.024710264900662332
rank_at_5_shap:  0.07820364238410596
rank_at_10_A_shap:  0.3541804635761585
rank_at_20_A_shap:  0.4008443708609273
rank_at_50_A_shap:  0.4800463576158946
rank_at_100_A_shap:  0.5565082781456964
rank_at_k_B_shap:  0.3058915282956655
NDCG_at_1_shap:  0.24710264900662252
NDCG_at_5_shap:  0.3051170776666
NDCG_at_10_shap:  0.3242669452721907
NDCG_at_20_shap:  0.3383976044688399
NDCG_at_50_shap:  0.35986574613520045
NDCG_at_100_shap:  0.37509433127174907


In [ ]:
names = ['POS_at_1_j_g', 'POS_at_5_j_g', 'POS_at_10_j_g', 'POS_at_20_j_g', 'POS_at_50_j_g', 'POS_at_100_j_g', 'NEG_at_1_j_g', 'NEG_at_5_j_g', 'NEG_at_10_j_g', 'NEG_at_20_j_g', 'NEG_at_50_j_g', 'NEG_at_100_j_g', 'users_DEL_j_g', 'users_INS_j_g', 'rank_at_1_j_g', 'rank_at_5_j_g', 'rank_at_10_A_j_g', 'rank_at_20_A_j_g', 'rank_at_50_A_j_g', 'rank_at_100_A_j_g', 'rank_at_k_B_j_g', 'NDCG_at_1_j_g', 'NDCG_at_5_j_g', 'NDCG_at_10_j_g', 'NDCG_at_20_j_g', 'NDCG_at_50_j_g', 'NDCG_at_100_j_g','POS_at_1_c_s', 'POS_at_5_c_s', 'POS_at_10_c_s', 'POS_at_20_c_s', 'POS_at_50_c_s', 'POS_at_100_c_s', 'NEG_at_1_c_s', 'NEG_at_5_c_s', 'NEG_at_10_c_s', 'NEG_at_20_c_s', 'NEG_at_50_c_s', 'NEG_at_100_c_s', 'users_DEL_c_s', 'users_INS_c_s', 'rank_at_1_c_s', 'rank_at_5_c_s', 'rank_at_10_A_c_s', 'rank_at_20_A_c_s', 'rank_at_50_A_c_s', 'rank_at_100_A_c_s', 'rank_at_k_B_c_s', 'NDCG_at_1_c_s', 'NDCG_at_5_c_s', 'NDCG_at_10_c_s', 'NDCG_at_20_c_s', 'NDCG_at_50_c_s', 'NDCG_at_100_c_s’,
'POS_at_1_lime', 'POS_at_5_lime', 'POS_at_10_lime', 'POS_at_20_lime', 'POS_at_50_lime', 'POS_at_100_lime', 'NEG_at_1_lime', 'NEG_at_5_lime', 'NEG_at_10_lime', 'NEG_at_20_lime', 'NEG_at_50_lime', 'NEG_at_100_lime', 'users_DEL_lime', 'users_INS_lime', 'rank_at_1_lime', 'rank_at_5_lime', 'rank_at_10_A_lime', 'rank_at_20_A_lime', 'rank_at_50_A_lime', 'rank_at_100_A_lime', 'rank_at_k_B_lime', 'NDCG_at_1_lime', 'NDCG_at_5_lime', 'NDCG_at_10_lime', 'NDCG_at_20_lime', 'NDCG_at_50_lime', 'NDCG_at_100_lime',
'POS_at_1_ltx', 'POS_at_5_ltx', 'POS_at_10_ltx', 'POS_at_20_ltx', 'POS_at_50_ltx', 'POS_at_100_ltx', 'NEG_at_1_ltx', 'NEG_at_5_ltx', 'NEG_at_10_ltx', 'NEG_at_20_ltx', 'NEG_at_50_ltx', 'NEG_at_100_ltx', 'users_DEL_ltx', 'users_INS_ltx', 'rank_at_1_ltx', 'rank_at_5_ltx', 'rank_at_10_A_ltx', 'rank_at_20_A_ltx', 'rank_at_50_A_ltx', 'rank_at_100_A_ltx', 'rank_at_k_B_ltx', 'NDCG_at_1_ltx', 'NDCG_at_5_ltx', 'NDCG_at_10_ltx', 'NDCG_at_20_ltx', 'NDCG_at_50_ltx', 'NDCG_at_100_ltx', 'POS_at_1_tf_idf', 'POS_at_5_tf_idf', 'POS_at_10_tf_idf', 'POS_at_20_tf_idf', 'POS_at_50_tf_idf', 'POS_at_100_tf_idf', 'NEG_at_1_tf_idf', 'NEG_at_5_tf_idf', 'NEG_at_10_tf_idf', 'NEG_at_20_tf_idf', 'NEG_at_50_tf_idf', 'NEG_at_100_tf_idf', 'users_DEL_tf_idf', 'users_INS_tf_idf', 'rank_at_1_tf_idf', 'rank_at_5_tf_idf', 'rank_at_10_A_tf_idf', 'rank_at_20_A_tf_idf', 'rank_at_50_A_tf_idf', 'rank_at_100_A_tf_idf', 'rank_at_k_B_tf_idf', 'NDCG_at_1_tf_idf', 'NDCG_at_5_tf_idf', 'NDCG_at_10_tf_idf', 'NDCG_at_20_tf_idf', 'NDCG_at_50_tf_idf', 'NDCG_at_100_tf_idf’,‘POS_at_1_pop', 'POS_at_5_pop', 'POS_at_10_pop', 'POS_at_20_pop', 'POS_at_50_pop', 'POS_at_100_pop', 'NEG_at_1_pop', 'NEG_at_5_pop', 'NEG_at_10_pop', 'NEG_at_20_pop', 'NEG_at_50_pop', 'NEG_at_100_pop', 'users_DEL_pop', 'users_INS_pop', 'rank_at_1_pop', 'rank_at_5_pop', 'rank_at_10_A_pop', 'rank_at_20_A_pop', 'rank_at_50_A_pop', 'rank_at_100_A_pop', 'rank_at_k_B_pop', 'NDCG_at_1_pop', 'NDCG_at_5_pop', 'NDCG_at_10_pop', 'NDCG_at_20_pop', 'NDCG_at_50_pop', 'NDCG_at_100_pop’]

values = [POS_at_1_j_g, pos_at_5_j_g, POS_at_10_j_g, POS_at_20_j_g, POS_at_50_j_g, POS_at_100_j_g, NEG_at_1_j_g, NEG_at_5_j_g, NEG_at_10_j_g, NEG_at_20_j_g, NEG_at_50_j_g, NEG_at_100_j_g, users_DEL_j_g, users_INS_j_g, rank_at_1_j_g, rank_at_5_j_g, rank_at_10_A_j_g, rank_at_20_A_j_g, rank_at_50_A_j_g, rank_at_100_A_j_g, rank_at_k_B_j_g, NDCG_at_1_j_g, NDCG_at_5_j_g, NDCG_at_10_j_g, NDCG_at_20_j_g, NDCG_at_50_j_g, NDCG_at_100_j_g, POS_at_1_c_s, pos_at_5_c_s, POS_at_10_c_s, POS_at_20_c_s, POS_at_50_c_s, POS_at_100_c_s, NEG_at_1_c_s, NEG_at_5_c_s, NEG_at_10_c_s, NEG_at_20_c_s, NEG_at_50_c_s, NEG_at_100_c_s, users_DEL_c_s, users_INS_c_s, rank_at_1_c_s, rank_at_5_c_s, rank_at_10_A_c_s, rank_at_20_A_c_s, rank_at_50_A_c_s, rank_at_100_A_c_s, rank_at_k_B_c_s, NDCG_at_1_c_s, NDCG_at_5_c_s, NDCG_at_10_c_s, NDCG_at_20_c_s, NDCG_at_50_c_s, NDCG_at_100_c_s,POS_at_1_lime, pos_at_5_lime, POS_at_10_lime, POS_at_20_lime, POS_at_50_lime, POS_at_100_lime, NEG_at_1_lime, NEG_at_5_lime, NEG_at_10_lime, NEG_at_20_lime, NEG_at_50_lime, NEG_at_100_lime, users_DEL_lime, users_INS_lime, rank_at_1_lime, rank_at_5_lime, rank_at_10_A_lime, rank_at_20_A_lime, rank_at_50_A_lime, rank_at_100_A_lime, rank_at_k_B_lime, NDCG_at_1_lime, NDCG_at_5_lime, NDCG_at_10_lime, NDCG_at_20_lime, NDCG_at_50_lime, NDCG_at_100_lime, POS_at_1_ltx, pos_at_5_ltx, POS_at_10_ltx, POS_at_20_ltx, POS_at_50_ltx, POS_at_100_ltx, NEG_at_1_ltx, NEG_at_5_ltx, NEG_at_10_ltx, NEG_at_20_ltx, NEG_at_50_ltx, NEG_at_100_ltx, users_DEL_ltx, users_INS_ltx, rank_at_1_ltx, rank_at_5_ltx, rank_at_10_A_ltx, rank_at_20_A_ltx, rank_at_50_A_ltx, rank_at_100_A_ltx, rank_at_k_B_ltx, NDCG_at_1_ltx, NDCG_at_5_ltx, NDCG_at_10_ltx, NDCG_at_20_ltx, NDCG_at_50_ltx, NDCG_at_100_ltx, POS_at_1_tf_idf, pos_at_5_tf_idf, POS_at_10_tf_idf, POS_at_20_tf_idf, POS_at_50_tf_idf, POS_at_100_tf_idf, NEG_at_1_tf_idf, NEG_at_5_tf_idf, NEG_at_10_tf_idf, NEG_at_20_tf_idf, NEG_at_50_tf_idf, NEG_at_100_tf_idf, users_DEL_tf_idf, users_INS_tf_idf, rank_at_1_tf_idf, rank_at_5_tf_idf, rank_at_10_A_tf_idf, rank_at_20_A_tf_idf, rank_at_50_A_tf_idf, rank_at_100_A_tf_idf, rank_at_k_B_tf_idf, NDCG_at_1_tf_idf, NDCG_at_5_tf_idf, NDCG_at_10_tf_idf, NDCG_at_20_tf_idf, NDCG_at_50_tf_idf, NDCG_at_100_tf_idf, POS_at_1_pop, pos_at_5_pop, POS_at_10_pop, POS_at_20_pop, POS_at_50_pop, POS_at_100_pop, NEG_at_1_pop, NEG_at_5_pop, NEG_at_10_pop, NEG_at_20_pop, NEG_at_50_pop, NEG_at_100_pop, users_DEL_pop, users_INS_pop, rank_at_1_pop, rank_at_5_pop, rank_at_10_A_pop, rank_at_20_A_pop, rank_at_50_A_pop, rank_at_100_A_pop, rank_at_k_B_pop, NDCG_at_1_pop, NDCG_at_5_pop, NDCG_at_10_pop, NDCG_at_20_pop, NDCG_at_50_pop, NDCG_at_100_pop]


d = {'names':names, 'values': values}

MLP_results_ml1m = pd.DataFrame(d)


normalized_values = []
for i in range(len(values)):
    lst = []
    for j in range(len(values[i])):
        lst.append(values[i][j]/a)
    normalized_values.append(lst)

MLP_results_ml1m['normalized_values'] = normalized_values

AUC = []
for i in range(len(values)):
    sublist = MLP_results_ml1m['normalized_values'][i][1:]
    AUC.append(np.mean(sublist))

MLP_results_ml1m['AUC'] = AUC

MLP_results_ml1m.head()

MLP_results_ml1m.to_csv('MLP_19_4_results_ml1m.csv')

In [ ]:
!pip freeze > requirements.txt